# Hamblin semantics introductory fragment
## Author: Kyle Rawlins
### Updated: 7/11/16

This is an implementation of Hamblin semantics.  It is currently quite incomplete, but walks through the basic principles.  It also may be useful for describing some lexicon manipulation techniques in the lambda notebook (see also the appendix).

In [1]:
reload_lamb()
%lambctl reset

In [2]:
type_e = types.type_e
type_t = types.type_t
type_n = types.type_n
type_s = types.BasicType("s")
ts = meta.get_type_system()
ts.add_atomic(type_s)
ts

The first thing to do is to 'hamblinize' the lexicon.  This amounts to converting ordinary meanings to singleton sets with their contents.

There are a number of options for how to implement this in a typed lambda calculus.  Here, we will exploit the fact that the Lambda Notebook has a type for sets.  If $\alpha$ is a type, then $\{\alpha\}$ is the type of sets containing elements of type $\alpha$.

The next bit of code defines a special operation `hamblinize` to simplify the process a bit; it can be safely ignored.

In [3]:
def hamblinize_te(te):
    """Hamblinize a single lexical item.  Do so by building a singleton set out of it."""
    if meta.get_type_system().unify(te.type, meta.tp("{X}")): 
        # assume this item is already hamblinized
        return te
    elif meta.get_type_system().unify(te.type, meta.tp("<{X},Y>")): 
        # heuristic: if function whose domain is a set of some sort, assume that this is a Hamblin operator.
        # may miss cases.  Better to just run this on content items...
        return te
    # wrap the content of the lexical item as a singleton set.
    return meta.ListedSet([te])    

#continuize_lex(cat1.content)
lamb.parsing.eq_transforms["hamblin"] = hamblinize_te

In [4]:
hamblinize_te(te("L x: Cat(x)"))

INFO (meta): Coerced guessed type for 'Cat_t' into <e,t>, to match argument 'x_e'


{(λ x_e: Cat_<e,t>(x_e))}

Let's see this in operation:

In [5]:
%%lamb
||cat|| =<hamblin> L x_e : L w_s : Cat(w,x)
||gray|| =<hamblin> L x_e : L w_s : Gray(w,x)
||john|| =<hamblin> John_e
x =<hamblin> L y_e : y
||test|| = L x_e : Test(x) # don't hamblinize this

INFO (meta): Coerced guessed type for 'Cat_t' into <(s,e),t>, to match argument '(w_s, x_e)'
INFO (meta): Coerced guessed type for 'Gray_t' into <(s,e),t>, to match argument '(w_s, x_e)'
INFO (meta): Coerced guessed type for 'Test_t' into <e,t>, to match argument 'x_e'


$[\![\mathbf{\text{cat}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\}} \:=\: $$\{\lambda{} x_{e} \: . \: \lambda{} w_{s} \: . \: {Cat}({w}, {x})\}$<br />
$[\![\mathbf{\text{gray}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\}} \:=\: $$\{\lambda{} x_{e} \: . \: \lambda{} w_{s} \: . \: {Gray}({w}, {x})\}$<br />
$[\![\mathbf{\text{john}}]\!]^{}_{\left\{e\right\}} \:=\: $$\{{John}_{e}\}$<br />
${x}_{\left\{\left\langle{}e,e\right\rangle{}\right\}}\:=\:\{\lambda{} y_{e} \: . \: {y}\}$<br />
$[\![\mathbf{\text{test}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Test}({x})$

Next, we need to add a way of composing entries *pointwise*.  The following code implements the most general case of Pointwise FA.

In the long run, one would want to automatically simplify these expressions; this is easy to do by hand, but it is a hard problem in the general case, so things are left mostly unsimplified.

In [6]:
pfa_combinator = %te L f_{<X,Y>} : L a_{X} : Set x_Y : Exists f1_<X,Y> : (Exists a1_X : (f1 << f & a1 << a) & x <=> f1(a1))

def pfa_combinator2(funtype, argtype):
    return te("L f_{%s} : L a_{%s} : Set x_%s : Exists f1_%s : (Exists a1_%s : (f1 << f & a1 << a) & x <=> f1(a1))" % (repr(funtype), repr(argtype), repr(funtype.right), repr(funtype), repr(argtype)))

pfa_combinator2(tp("<e,<s,t>>"), types.type_e)
pfa_combinator

(λ f_{<I37,Y>}: (λ a_{I37}: (Set x_Y: (Exists f1_<I37,Y>: (Exists a1_I37: (((f1_<I37,Y> << f_{<I37,Y>}) & (a1_I37 << a_{I37})) & (x_Y <=> f1_<I37,Y>(a1_I40))))))))

In [7]:
def pfa_worstcase(fun, arg):
    ts = meta.get_type_system()
    if not (isinstance(fun.type, types.SetType) 
            and isinstance(arg.type, types.SetType)):
        raise types.TypeMismatch(fun, arg, "Pointwise Function Application")
    if not (ts.fun_arg_check_types_bool(fun.type.content_type, arg.type.content_type)):
        raise types.TypeMismatch(fun, arg, "Pointwise Function Application")
    return pfa_combinator(fun.type.content_type, arg.type.content_type)(fun)(arg)

system = lang.td_system.copy()
#system.add_rule(lang.binary_factory(pfa_worstcase, "PFA"))
system.add_binary_rule(pfa_combinator, "PFA")
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [8]:
john * cat

CompositionResult(results=[⟦[cat john]⟧ = (Set x_<s,t>: (Exists f1_<e,<s,t>>: (Exists a1_e: (((f1_<e,<s,t>> << {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}) & (a1_e << {John_e})) & (x_<s,t> <=> f1_<e,<s,t>>(a1_e))))))], failures=[⟦[john cat]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦cat⟧ = {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}'/{<e,<s,t>>} conflict (mode: Function Application), ⟦[cat john]⟧ = Type mismatch: '⟦cat⟧ = {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}'/{<e,<s,t>>} and '⟦john⟧ = {John_e}'/{e} conflict (mode: Function Application), ⟦[john cat]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦cat⟧ = {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}'/{<e,<s,t>>} conflict (mode: Predicate Modification), ⟦[john cat]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦cat⟧ = {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}'/{<e,<s,t>>} conflict (mode: Predicate Abstraction), ⟦[cat john]⟧ = Type mismatch: '⟦cat⟧ = {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}'/{<e,<s,t>>} and '⟦john⟧ = {John_e}'/{e} conflict (mode: Predicate Abstraction), ⟦[john cat]⟧ = Type mismatch: '(λ f_{<I37,Y>}: (λ a_{I37}: (Set x_Y: (Exists f1_<I37,Y>: (Exists a1_I37: (((f1_<I37,Y> << f_{<I37,Y>}) & (a1_I37 << a_{I37})) & (x_Y <=> f1_<I37,Y>(a1_I40))))))))'/<{<I37,Y>},<{I37},{Y}>> and '{John_e}'/{e} conflict (mode: Function argument combination (unification failed))])

Instead of automatically simplifying things, I'll exploit the fact that sets are represented differently (as a list) if they are finite, so we can special-case PFA for ListedSets, and fall back on the combinator for the general case.

Note that some limited reduction does occur with the $\in$ operator; $x \in \{y \:|\: \phi\}$ gets converted to $(\lambda y : \phi)(x)$ and reduced, illustrated in the next cell.

In [9]:
r = te("x_e << (Set y_e : Test_<e,t>(y))")
r.reduce_all().derivation

1. (x_e << (Set y_e: Test_<e,t>(y_e)))
 2. Test_<e,t>(x_e)    (∈ reduction)

In [10]:
def pfa_listed(fun, arg):
    result = list()
    for felem in fun.args:
        for aelem in arg.args:
            result.append(felem(aelem))
    return meta.ListedSet(result)

def pfa_general(fun, arg):
    ts = meta.get_type_system()
    general = pfa_combinator(fun)(arg) # do this for type checking
    if isinstance(fun, meta.ListedSet) and isinstance(arg, meta.ListedSet):
        return pfa_listed(fun, arg)
    else:
        return general.reduce_all()
    
system = lang.td_system.copy()
system.add_binary_rule_uncurried(pfa_general, "PFA")
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [11]:
john * cat

CompositionResult(results=[⟦[cat john]⟧ = {(λ w_s: Cat_<(s,e),t>(w_s, John_e))}], failures=[⟦[john cat]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦cat⟧ = {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}'/{<e,<s,t>>} conflict (mode: Function Application), ⟦[cat john]⟧ = Type mismatch: '⟦cat⟧ = {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}'/{<e,<s,t>>} and '⟦john⟧ = {John_e}'/{e} conflict (mode: Function Application), ⟦[john cat]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦cat⟧ = {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}'/{<e,<s,t>>} conflict (mode: Predicate Modification), ⟦[john cat]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦cat⟧ = {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}'/{<e,<s,t>>} conflict (mode: Predicate Abstraction), ⟦[cat john]⟧ = Type mismatch: '⟦cat⟧ = {(λ x_e: (λ w_s: Cat_<(s,e),t>(w_s, x_e)))}'/{<e,<s,t>>} and '⟦john⟧ = {John_e}'/{e} conflict (mode: Predicate Abstraction), ⟦[john cat]⟧ = Type mismatch: '(λ f_{<I37,Y>}: (λ a_{I37}: (Set x_Y: (Exists f1_<I37,Y>: (Exists a1_I37: (((f1_<I37,Y> << f_{<I37,Y>}) & (a1_I37 << a_{I37})) & (x_Y <=> f1_<I37,Y>(a1_I40))))))))'/<{<I37,Y>},<{I37},{Y}>> and '{John_e}'/{e} conflict (mode: Function argument combination (unification failed))])

In [12]:
%%lamb
## To simplify, let's take there to only be three human-like entities in the universe.
||who|| = {John_e, Mary_e, Sue_e}
||saw|| =<hamblin> L x_e : L y_e : L w_s : Saw(w,y,x)

INFO (meta): Coerced guessed type for 'Saw_t' into <(s,e,e),t>, to match argument '(w_s, y_e, x_e)'


$[\![\mathbf{\text{who}}]\!]^{}_{\left\{e\right\}} \:=\: $$\{{John}_{e}, {Mary}_{e}, {Sue}_{e}\}$<br />
$[\![\mathbf{\text{saw}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\rangle{}\right\}} \:=\: $$\{\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: \lambda{} w_{s} \: . \: {Saw}({w}, {y}, {x})\}$

In [13]:
(cat * who).tree()

1 composition path:<br /><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{cat}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{\lambda{} x_{e} \: . \: \lambda{} w_{s} \: . \: {Cat}({w}, {x})\}$</span></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{who}}]\!]^{}_{\left\{e\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{{John}_{e}, {Mary}_{e}, {Sue}_{e}\}$</span></div></div></div></div></div></div></div><div style="display:table-cell;vertical-align:middle;border-left:1px solid #848482;padding:0.5em"><div style="white-space:nowrap; color:blue;"><b>[<span>PFA</span>]</b></div></div></div><div align="center" style="display:table-row;padding: 5px;"><div align="center" style="display:table-cell;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{[cat who]}}]\!]^{}_{\left\{\left\langle{}s,t\right\rangle{}\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{\lambda{} w_{s} \: . \: {Cat}({w}, {John}_{e}), \lambda{} w_{s} \: . \: {Cat}({w}, {Mary}_{e}), \lambda{} w_{s} \: . \: {Cat}({w}, {Sue}_{e})\}$</span></div></div></div></div></div><div style="display:table-cell;"></div></div></div><br /><br />

Wh-items compose in situ.  Intuitively, this leads to sets that percolate up the tree.

In [14]:
john * (saw * who)

CompositionResult(results=[⟦[[saw who] john]⟧ = {(λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Mary_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Sue_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e))}], failures=[⟦[john [saw who]]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦[saw who]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, Sue_e))), (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, Mary_e))), (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} conflict (mode: Function Application), ⟦[[saw who] john]⟧ = Type mismatch: '⟦[saw who]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, Sue_e))), (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, Mary_e))), (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} and '⟦john⟧ = {John_e}'/{e} conflict (mode: Function Application), ⟦[john [saw who]]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦[saw who]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, Sue_e))), (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, Mary_e))), (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} conflict (mode: Predicate Modification), ⟦[john [saw who]]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦[saw who]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, Sue_e))), (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, Mary_e))), (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} conflict (mode: Predicate Abstraction), ⟦[[saw who] john]⟧ = Type mismatch: '⟦[saw who]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, Sue_e))), (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, Mary_e))), (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} and '⟦john⟧ = {John_e}'/{e} conflict (mode: Predicate Abstraction), ⟦[john [saw who]]⟧ = Type mismatch: '(λ f_{<I37,Y>}: (λ a_{I37}: (Set x_Y: (Exists f1_<I37,Y>: (Exists a1_I37: (((f1_<I37,Y> << f_{<I37,Y>}) & (a1_I37 << a_{I37})) & (x_Y <=> f1_<I37,Y>(a1_I40))))))))'/<{<I37,Y>},<{I37},{Y}>> and '{John_e}'/{e} conflict (mode: Function argument combination (unification failed))])

In [15]:
(john * (saw * who)).tree()

1 composition path:<br /><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{saw}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\rangle{}\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: \lambda{} w_{s} \: . \: {Saw}({w}, {y}, {x})\}$</span></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{who}}]\!]^{}_{\left\{e\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{{John}_{e}, {Mary}_{e}, {Sue}_{e}\}$</span></div></div></div></div></div></div></div><div style="display:table-cell;vertical-align:middle;border-left:1px solid #848482;padding:0.5em"><div style="white-space:nowrap; color:blue;"><b>[<span>PFA</span>]</b></div></div></div><div align="center" style="display:table-row;padding: 5px;"><div align="center" style="display:table-cell;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{[saw who]}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{\lambda{} y_{e} \: . \: \lambda{} w_{s} \: . \: {Saw}({w}, {y}, {John}_{e}), \lambda{} y_{e} \: . \: \lambda{} w_{s} \: . \: {Saw}({w}, {y}, {Mary}_{e}), \lambda{} y_{e} \: . \: \lambda{} w_{s} \: . \: {Saw}({w}, {y}, {Sue}_{e})\}$</span></div></div></div></div></div><div style="display:table-cell;"></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{john}}]\!]^{}_{\left\{e\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{{John}_{e}\}$</span></div></div></div></div></div></div></div><div style="display:table-cell;vertical-align:middle;border-left:1px solid #848482;padding:0.5em"><div style="w

In [16]:
who * (saw * john)

CompositionResult(results=[⟦[[saw john] who]⟧ = {(λ w_s: Saw_<(s,e,e),t>(w_s, Mary_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, Sue_e, John_e))}], failures=[⟦[who [saw john]]⟧ = Type mismatch: '⟦who⟧ = {Mary_e, Sue_e, John_e}'/{e} and '⟦[saw john]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} conflict (mode: Function Application), ⟦[[saw john] who]⟧ = Type mismatch: '⟦[saw john]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} and '⟦who⟧ = {Mary_e, Sue_e, John_e}'/{e} conflict (mode: Function Application), ⟦[who [saw john]]⟧ = Type mismatch: '⟦who⟧ = {Mary_e, Sue_e, John_e}'/{e} and '⟦[saw john]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} conflict (mode: Predicate Modification), ⟦[who [saw john]]⟧ = Type mismatch: '⟦who⟧ = {Mary_e, Sue_e, John_e}'/{e} and '⟦[saw john]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} conflict (mode: Predicate Abstraction), ⟦[[saw john] who]⟧ = Type mismatch: '⟦[saw john]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} and '⟦who⟧ = {Mary_e, Sue_e, John_e}'/{e} conflict (mode: Predicate Abstraction), ⟦[who [saw john]]⟧ = Type mismatch: '(λ f_{<I37,Y>}: (λ a_{I37}: (Set x_Y: (Exists f1_<I37,Y>: (Exists a1_I37: (((f1_<I37,Y> << f_{<I37,Y>}) & (a1_I37 << a_{I37})) & (x_Y <=> f1_<I37,Y>(a1_I40))))))))'/<{<I37,Y>},<{I37},{Y}>> and '{Mary_e, Sue_e, John_e}'/{e} conflict (mode: Function argument combination (unification failed))])

The analysis of existentials in a compositional Hamblin semantics (e.g. for Japanese) can be illustrated in this simple example by adding Hamblin 1-place quantificational operators that compose via ordinary FA.

In [17]:
%%lamb
||HExists|| = L p_{<s,t>} : {(Lambda w_s  : Exists q_<s,t> : q(w) & (q << p))}
||HForall|| = L p_{<s,t>} : {(Lambda w_s  : Forall q_<s,t> : q(w) >> (q << p))}

$[\![\mathbf{\text{HExists}}]\!]^{}_{\left\langle{}\left\{\left\langle{}s,t\right\rangle{}\right\},\left\{\left\langle{}s,t\right\rangle{}\right\}\right\rangle{}} \:=\: $$\lambda{} p_{\left\{\left\langle{}s,t\right\rangle{}\right\}} \: . \: \{\lambda{} w_{s} \: . \: \exists{} q_{\left\langle{}s,t\right\rangle{}} \: . \: ({q}({w}) \wedge{} ({q} \in{} {p}))\}$<br />
$[\![\mathbf{\text{HForall}}]\!]^{}_{\left\langle{}\left\{\left\langle{}s,t\right\rangle{}\right\},\left\{\left\langle{}s,t\right\rangle{}\right\}\right\rangle{}} \:=\: $$\lambda{} p_{\left\{\left\langle{}s,t\right\rangle{}\right\}} \: . \: \{\lambda{} w_{s} \: . \: \forall{} q_{\left\langle{}s,t\right\rangle{}} \: . \: ({q}({w}) \rightarrow{} ({q} \in{} {p}))\}$

In [18]:
HExists * (who * (saw * john))

CompositionResult(results=[⟦[HExists [[saw john] who]]⟧ = {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << {(λ w_s: Saw_<(s,e,e),t>(w_s, Mary_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, Sue_e, John_e))}))))}], failures=[⟦[[[saw john] who] HExists]⟧ = Type mismatch: '⟦[[saw john] who]⟧ = {(λ w_s: Saw_<(s,e,e),t>(w_s, Mary_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, Sue_e, John_e))}'/{<s,t>} and '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Function Application), ⟦[HExists [[saw john] who]]⟧ = Type mismatch: '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> and '⟦[[saw john] who]⟧ = {(λ w_s: Saw_<(s,e,e),t>(w_s, Mary_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, Sue_e, John_e))}'/{<s,t>} conflict (mode: Predicate Modification), ⟦[HExists [[saw john] who]]⟧ = Type mismatch: '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> and '⟦[[saw john] who]⟧ = {(λ w_s: Saw_<(s,e,e),t>(w_s, Mary_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, Sue_e, John_e))}'/{<s,t>} conflict (mode: Predicate Abstraction), ⟦[[[saw john] who] HExists]⟧ = Type mismatch: '⟦[[saw john] who]⟧ = {(λ w_s: Saw_<(s,e,e),t>(w_s, Mary_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, Sue_e, John_e))}'/{<s,t>} and '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Predicate Abstraction), ⟦[HExists [[saw john] who]]⟧ = Type mismatch: '(λ f_{<I37,Y>}: (λ a_{I37}: (Set x_Y: (Exists f1_<I37,Y>: (Exists a1_I37: (((f1_<I37,Y> << f_{<I37,Y>}) & (a1_I37 << a_{I37})) & (x_Y <=> f1_<I37,Y>(a1_I40))))))))'/<{<I37,Y>},<{I37},{Y}>> and '(λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Function argument combination (unification failed)), ⟦[[[saw john] who] HExists]⟧ = Type mismatch: '((λ f_{<s,t>}: (λ a_{s}: (Set x_t: (Exists f1_<s,t>: (Exists a1_s: (((f1_<s,t> << f_{<s,t>}) & (a1_s << a_{s})) & (x_t <=> f1_<s,t>(a1_s)))))))))({(λ w_s: Saw_<(s,e,e),t>(w_s, Mary_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, Sue_e, John_e))})'/<{s},{t}> and '(λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Function argument combination (unification failed))])

In [19]:
HExists * (john * (saw * who))

CompositionResult(results=[⟦[HExists [[saw who] john]]⟧ = {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << {(λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Mary_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Sue_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e))}))))}], failures=[⟦[[[saw who] john] HExists]⟧ = Type mismatch: '⟦[[saw who] john]⟧ = {(λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Mary_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Sue_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e))}'/{<s,t>} and '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Function Application), ⟦[HExists [[saw who] john]]⟧ = Type mismatch: '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> and '⟦[[saw who] john]⟧ = {(λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Mary_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Sue_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e))}'/{<s,t>} conflict (mode: Predicate Modification), ⟦[HExists [[saw who] john]]⟧ = Type mismatch: '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> and '⟦[[saw who] john]⟧ = {(λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Mary_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Sue_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e))}'/{<s,t>} conflict (mode: Predicate Abstraction), ⟦[[[saw who] john] HExists]⟧ = Type mismatch: '⟦[[saw who] john]⟧ = {(λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Mary_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Sue_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e))}'/{<s,t>} and '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Predicate Abstraction), ⟦[HExists [[saw who] john]]⟧ = Type mismatch: '(λ f_{<I37,Y>}: (λ a_{I37}: (Set x_Y: (Exists f1_<I37,Y>: (Exists a1_I37: (((f1_<I37,Y> << f_{<I37,Y>}) & (a1_I37 << a_{I37})) & (x_Y <=> f1_<I37,Y>(a1_I40))))))))'/<{<I37,Y>},<{I37},{Y}>> and '(λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Function argument combination (unification failed)), ⟦[[[saw who] john] HExists]⟧ = Type mismatch: '((λ f_{<s,t>}: (λ a_{s}: (Set x_t: (Exists f1_<s,t>: (Exists a1_s: (((f1_<s,t> << f_{<s,t>}) & (a1_s << a_{s})) & (x_t <=> f1_<s,t>(a1_s)))))))))({(λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Mary_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, Sue_e)), (λ w_s: Saw_<(s,e,e),t>(w_s, John_e, John_e))})'/<{s},{t}> and '(λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Function argument combination (unification failed))])

In [20]:
(HExists * (john * (saw * who))).tree()

1 composition path:<br /><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{HExists}}]\!]^{}_{\left\langle{}\left\{\left\langle{}s,t\right\rangle{}\right\},\left\{\left\langle{}s,t\right\rangle{}\right\}\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} p_{\left\{\left\langle{}s,t\right\rangle{}\right\}} \: . \: \{\lambda{} w_{s} \: . \: \exists{} q_{\left\langle{}s,t\right\rangle{}} \: . \: ({q}({w}) \wedge{} ({q} \in{} {p}))\}$</span></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{saw}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\rangle{}\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: \lambda{} w_{s} \: . \: {Saw}({w}, {y}, {x})\}$</span></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{who}}]\!]^{}_{\left\{e\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{{John}_{e}, {Mary}_{e}, {Sue}_{e}\}$</span></div></div></div></div></div></div></div><div style="display:table-cell;vertical-align:middle;border-left:1px solid #848482;padding:0.5em"><div style="white-space:nowrap; color:blue;"><b>[<span>PFA</span>]</b></div></div></div><div align="center" style="display:table-row;padding: 5px;"><div align="center" style="display:table-cell;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{[saw who]}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}s,t\right\ra

More generally, the listed versions of this won't do, because we don't want to assume that the relevant sets are finite.  Or in other words, we don't want to define 'who' extensionally:

In [21]:
%%lamb
||who|| = Set x_e : Human(x)

INFO (meta): Coerced guessed type for 'Human_t' into <e,t>, to match argument 'x_e'


$[\![\mathbf{\text{who}}]\!]^{}_{\left\{e\right\}} \:=\: $$\{x_{e}\:|\: {Human}({x}_{e})\}$

At the moment the lambda notebook doesn't do much simplification of these expressions, so they will be a bit unwieldy.

In [22]:
(cat * who).tree()

1 composition path:<br /><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{cat}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{\lambda{} x_{e} \: . \: \lambda{} w_{s} \: . \: {Cat}({w}, {x})\}$</span></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{who}}]\!]^{}_{\left\{e\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{x_{e}\:|\: {Human}({x}_{e})\}$</span></div></div></div></div></div></div></div><div style="display:table-cell;vertical-align:middle;border-left:1px solid #848482;padding:0.5em"><div style="white-space:nowrap; color:blue;"><b>[<span>PFA</span>]</b></div></div></div><div align="center" style="display:table-row;padding: 5px;"><div align="center" style="display:table-cell;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{[cat who]}}]\!]^{}_{\left\{\left\langle{}s,t\right\rangle{}\right\}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\{x_{\left\langle{}s,t\right\rangle{}}\:|\: \exists{} f1_{\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}} \: . \: \exists{} a1_{e} \: . \: ((({f1} \in{} \{\lambda{} x_{e} \: . \: \lambda{} w_{s} \: . \: {Cat}({w}, {x})\}) \wedge{} {Human}({a1})) \wedge{} ({x}_{\left\langle{}s,t\right\rangle{}} = {f1}({a1})))\}$</span></div></div></div></div></div><div style="display:table-cell;"></div></div></div><br /><br />

In [23]:
john * (saw * who)

CompositionResult(results=[⟦[[saw who] john]⟧ = (Set x_<s,t>: (Exists f1_<e,<s,t>>: (Exists a1_e: (((Exists f2_<e,<e,<s,t>>>: (Exists a1_e: (((f2_<e,<e,<s,t>>> << {(λ x_e: (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, x_e))))}) & Human_<e,t>(a1_e)) & (f1_<e,<s,t>> <=> f2_<e,<e,<s,t>>>(a1_e))))) & (a1_e << {John_e})) & (x_<s,t> <=> f1_<e,<s,t>>(a1_e))))))], failures=[⟦[john [saw who]]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦[saw who]⟧ = (Set x_<e,<s,t>>: (Exists f1_<e,<e,<s,t>>>: (Exists a1_e: (((f1_<e,<e,<s,t>>> << {(λ x_e: (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, x_e))))}) & Human_<e,t>(a1_e)) & (x_<e,<s,t>> <=> f1_<e,<e,<s,t>>>(a1_e))))))'/{<e,<s,t>>} conflict (mode: Function Application), ⟦[[saw who] john]⟧ = Type mismatch: '⟦[saw who]⟧ = (Set x_<e,<s,t>>: (Exists f1_<e,<e,<s,t>>>: (Exists a1_e: (((f1_<e,<e,<s,t>>> << {(λ x_e: (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, x_e))))}) & Human_<e,t>(a1_e)) & (x_<e,<s,t>> <=> f1_<e,<e,<s,t>>>(a1_e))))))'/{<e,<s,t>>} and '⟦john⟧ = {John_e}'/{e} conflict (mode: Function Application), ⟦[john [saw who]]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦[saw who]⟧ = (Set x_<e,<s,t>>: (Exists f1_<e,<e,<s,t>>>: (Exists a1_e: (((f1_<e,<e,<s,t>>> << {(λ x_e: (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, x_e))))}) & Human_<e,t>(a1_e)) & (x_<e,<s,t>> <=> f1_<e,<e,<s,t>>>(a1_e))))))'/{<e,<s,t>>} conflict (mode: Predicate Modification), ⟦[john [saw who]]⟧ = Type mismatch: '⟦john⟧ = {John_e}'/{e} and '⟦[saw who]⟧ = (Set x_<e,<s,t>>: (Exists f1_<e,<e,<s,t>>>: (Exists a1_e: (((f1_<e,<e,<s,t>>> << {(λ x_e: (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, x_e))))}) & Human_<e,t>(a1_e)) & (x_<e,<s,t>> <=> f1_<e,<e,<s,t>>>(a1_e))))))'/{<e,<s,t>>} conflict (mode: Predicate Abstraction), ⟦[[saw who] john]⟧ = Type mismatch: '⟦[saw who]⟧ = (Set x_<e,<s,t>>: (Exists f1_<e,<e,<s,t>>>: (Exists a1_e: (((f1_<e,<e,<s,t>>> << {(λ x_e: (λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, x_e))))}) & Human_<e,t>(a1_e)) & (x_<e,<s,t>> <=> f1_<e,<e,<s,t>>>(a1_e))))))'/{<e,<s,t>>} and '⟦john⟧ = {John_e}'/{e} conflict (mode: Predicate Abstraction), ⟦[john [saw who]]⟧ = Type mismatch: '(λ f_{<I37,Y>}: (λ a_{I37}: (Set x_Y: (Exists f1_<I37,Y>: (Exists a1_I37: (((f1_<I37,Y> << f_{<I37,Y>}) & (a1_I37 << a_{I37})) & (x_Y <=> f1_<I37,Y>(a1_I40))))))))'/<{<I37,Y>},<{I37},{Y}>> and '{John_e}'/{e} conflict (mode: Function argument combination (unification failed))])

In [24]:
who * (saw * john)

CompositionResult(results=[⟦[[saw john] who]⟧ = (Set x_<s,t>: (Exists f1_<e,<s,t>>: (Exists a1_e: (((f1_<e,<s,t>> << {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}) & Human_<e,t>(a1_e)) & (x_<s,t> <=> f1_<e,<s,t>>(a1_e))))))], failures=[⟦[who [saw john]]⟧ = Type mismatch: '⟦who⟧ = (Set x_e: Human_<e,t>(x_e))'/{e} and '⟦[saw john]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} conflict (mode: Function Application), ⟦[[saw john] who]⟧ = Type mismatch: '⟦[saw john]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} and '⟦who⟧ = (Set x_e: Human_<e,t>(x_e))'/{e} conflict (mode: Function Application), ⟦[who [saw john]]⟧ = Type mismatch: '⟦who⟧ = (Set x_e: Human_<e,t>(x_e))'/{e} and '⟦[saw john]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} conflict (mode: Predicate Modification), ⟦[who [saw john]]⟧ = Type mismatch: '⟦who⟧ = (Set x_e: Human_<e,t>(x_e))'/{e} and '⟦[saw john]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} conflict (mode: Predicate Abstraction), ⟦[[saw john] who]⟧ = Type mismatch: '⟦[saw john]⟧ = {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}'/{<e,<s,t>>} and '⟦who⟧ = (Set x_e: Human_<e,t>(x_e))'/{e} conflict (mode: Predicate Abstraction), ⟦[who [saw john]]⟧ = Type mismatch: '(λ f_{<I37,Y>}: (λ a_{I37}: (Set x_Y: (Exists f1_<I37,Y>: (Exists a1_I37: (((f1_<I37,Y> << f_{<I37,Y>}) & (a1_I37 << a_{I37})) & (x_Y <=> f1_<I37,Y>(a1_I40))))))))'/<{<I37,Y>},<{I37},{Y}>> and '(Set x_e: Human_<e,t>(x_e))'/{e} conflict (mode: Function argument combination (unification failed))])

In [25]:
HExists * (who * (saw * john))

CompositionResult(results=[⟦[HExists [[saw john] who]]⟧ = {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (Exists f1_<e,<s,t>>: (Exists a1_e: (((f1_<e,<s,t>> << {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}) & Human_<e,t>(a1_e)) & (q_<s,t> <=> f1_<e,<s,t>>(a1_e))))))))}], failures=[⟦[[[saw john] who] HExists]⟧ = Type mismatch: '⟦[[saw john] who]⟧ = (Set x_<s,t>: (Exists f1_<e,<s,t>>: (Exists a1_e: (((f1_<e,<s,t>> << {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}) & Human_<e,t>(a1_e)) & (x_<s,t> <=> f1_<e,<s,t>>(a1_e))))))'/{<s,t>} and '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Function Application), ⟦[HExists [[saw john] who]]⟧ = Type mismatch: '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> and '⟦[[saw john] who]⟧ = (Set x_<s,t>: (Exists f1_<e,<s,t>>: (Exists a1_e: (((f1_<e,<s,t>> << {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}) & Human_<e,t>(a1_e)) & (x_<s,t> <=> f1_<e,<s,t>>(a1_e))))))'/{<s,t>} conflict (mode: Predicate Modification), ⟦[HExists [[saw john] who]]⟧ = Type mismatch: '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> and '⟦[[saw john] who]⟧ = (Set x_<s,t>: (Exists f1_<e,<s,t>>: (Exists a1_e: (((f1_<e,<s,t>> << {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}) & Human_<e,t>(a1_e)) & (x_<s,t> <=> f1_<e,<s,t>>(a1_e))))))'/{<s,t>} conflict (mode: Predicate Abstraction), ⟦[[[saw john] who] HExists]⟧ = Type mismatch: '⟦[[saw john] who]⟧ = (Set x_<s,t>: (Exists f1_<e,<s,t>>: (Exists a1_e: (((f1_<e,<s,t>> << {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}) & Human_<e,t>(a1_e)) & (x_<s,t> <=> f1_<e,<s,t>>(a1_e))))))'/{<s,t>} and '⟦HExists⟧ = (λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Predicate Abstraction), ⟦[HExists [[saw john] who]]⟧ = Type mismatch: '(λ f_{<I37,Y>}: (λ a_{I37}: (Set x_Y: (Exists f1_<I37,Y>: (Exists a1_I37: (((f1_<I37,Y> << f_{<I37,Y>}) & (a1_I37 << a_{I37})) & (x_Y <=> f1_<I37,Y>(a1_I40))))))))'/<{<I37,Y>},<{I37},{Y}>> and '(λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Function argument combination (unification failed)), ⟦[[[saw john] who] HExists]⟧ = Type mismatch: '((λ f_{<s,t>}: (λ a_{s}: (Set x_t: (Exists f1_<s,t>: (Exists a1_s: (((f1_<s,t> << f_{<s,t>}) & (a1_s << a_{s})) & (x_t <=> f1_<s,t>(a1_s)))))))))((Set x_<s,t>: (Exists f1_<e,<s,t>>: (Exists a1_e: (((f1_<e,<s,t>> << {(λ y_e: (λ w_s: Saw_<(s,e,e),t>(w_s, y_e, John_e)))}) & Human_<e,t>(a1_e)) & (x_<s,t> <=> f1_<e,<s,t>>(a1_e)))))))'/<{s},{t}> and '(λ p_{<s,t>}: {(λ w_s: (Exists q_<s,t>: (q_<s,t>(w_s) & (q_<s,t> << p_{<s,t>}))))})'/<{<s,t>},{<s,t>}> conflict (mode: Function argument combination (unification failed))])

The above formulas are somewhat involved, so it is slightly more convenient to simply define a toy example with a finite set of individuals:

### Appendix: another technique for Hamblinization

Another way of Hamblinizing a lexicon would be to write extra magics for converting whole lexicons at once.  Here's a sketch of how to do this.

In [26]:
def hamblinize_item(item):
    """Hamblinize a single lexical item.  Do so by building a singleton set out of it."""
    if meta.ts_compatible(item.type, meta.tp("{?}")): #isinstance(item.type, types.SetType):
        # assume this item is already hamblinized
        return item
    elif meta.ts_compatible(item.type, meta.tp("<{?},?>")): #item.type.functional() and isinstance(item.type.left, types.SetType):
        # heuristic: if function whose domain is a set of some sort, assume that this is a Hamblin operator.
        # may miss cases.  Better to just run this on content items...
        return item
    new_i = item.copy()
    # wrap the content of the lexical item as a singleton set.
    new_i.content = meta.ListedSet([item.content])
    return new_i

# in the following two magics, variables that are not lexical items are ignored.  To change this, modify the else case above.
def h_magic(self, accum):
    """Hamblinize the accumulated definitions from a single cell, as a post-processing step"""
    new_accum = lamb.magics.process_items(hamblinize_item, accum)[0]
    for k in new_accum.keys():
        self.env[k] = new_accum[k]
    return new_accum

def h_magic_env(self):
    """Hamblinize the entire env"""
    self.env = lamb.magics.process_items(hamblinize_item, self.env)[0] # hamblinize every variable
    self.shell.push(self.env) # export the new variables to the interactive shell
    return parsing.latex_output(self.env, self.env)

lamb.magics.LambMagics.specials_post["hamblinize"] = h_magic
lamb.magics.LambMagics.specials["hamblinize_all"] = h_magic_env

In [27]:
%%lamb hamblinize
||cat|| = L x_e : L w_s : Cat(w,x)
||gray|| = L x_e : L w_s : Gray(w,x)
||john|| = J_e
x = L y_e : y

INFO (meta): Coerced guessed type for 'Cat_t' into <(s,e),t>, to match argument '(w_s, x_e)'
INFO (meta): Coerced guessed type for 'Gray_t' into <(s,e),t>, to match argument '(w_s, x_e)'


$[\![\mathbf{\text{cat}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\}} \:=\: $$\{\lambda{} x_{e} \: . \: \lambda{} w_{s} \: . \: {Cat}({w}, {x})\}$<br />
$[\![\mathbf{\text{gray}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\}} \:=\: $$\{\lambda{} x_{e} \: . \: \lambda{} w_{s} \: . \: {Gray}({w}, {x})\}$<br />
$[\![\mathbf{\text{john}}]\!]^{}_{\left\{e\right\}} \:=\: $$\{{J}_{e}\}$<br />
${x}_{\left\langle{}e,e\right\rangle{}}\:=\:\lambda{} y_{e} \: . \: {y}$

In [28]:
%%lamb
||test|| = L x_e : Test(x)

INFO (meta): Coerced guessed type for 'Test_t' into <e,t>, to match argument 'x_e'


$[\![\mathbf{\text{test}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Test}({x})$

In [29]:
%lambctl hamblinize_all

$[\![\mathbf{\text{cat}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\}} \:=\: $$\{\lambda{} x_{e} \: . \: \lambda{} w_{s} \: . \: {Cat}({w}, {x})\}$<br />
$[\![\mathbf{\text{gray}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\}} \:=\: $$\{\lambda{} x_{e} \: . \: \lambda{} w_{s} \: . \: {Gray}({w}, {x})\}$<br />
$[\![\mathbf{\text{john}}]\!]^{}_{\left\{e\right\}} \:=\: $$\{{J}_{e}\}$<br />
${x}_{\left\langle{}e,e\right\rangle{}}\:=\:\lambda{} y_{e} \: . \: {y}$<br />
$[\![\mathbf{\text{test}}]\!]^{}_{\left\{\left\langle{}e,t\right\rangle{}\right\}} \:=\: $$\{\lambda{} x_{e} \: . \: {Test}({x})\}$<br />
$[\![\mathbf{\text{who}}]\!]^{}_{\left\{e\right\}} \:=\: $$\{x_{e}\:|\: {Human}({x}_{e})\}$<br />
$[\![\mathbf{\text{saw}}]\!]^{}_{\left\{\left\langle{}e,\left\langle{}e,\left\langle{}s,t\right\rangle{}\right\rangle{}\right\rangle{}\right\}} \:=\: $$\{\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: \lambda{} w_{s} \: . \: {Saw}({w}, {y}, {x})\}$<br />
$[\![\mathbf{\text{HExists}}]\!]^{}_{\left\langle{}\left\{\left\langle{}s,t\right\rangle{}\right\},\left\{\left\langle{}s,t\right\rangle{}\right\}\right\rangle{}} \:=\: $$\lambda{} p_{\left\{\left\langle{}s,t\right\rangle{}\right\}} \: . \: \{\lambda{} w_{s} \: . \: \exists{} q_{\left\langle{}s,t\right\rangle{}} \: . \: ({q}({w}) \wedge{} ({q} \in{} {p}))\}$<br />
$[\![\mathbf{\text{HForall}}]\!]^{}_{\left\langle{}\left\{\left\langle{}s,t\right\rangle{}\right\},\left\{\left\langle{}s,t\right\rangle{}\right\}\right\rangle{}} \:=\: $$\lambda{} p_{\left\{\left\langle{}s,t\right\rangle{}\right\}} \: . \: \{\lambda{} w_{s} \: . \: \forall{} q_{\left\langle{}s,t\right\rangle{}} \: . \: ({q}({w}) \rightarrow{} ({q} \in{} {p}))\}$